# Day 15: Beverage Bandits


## Part 1

### Read the input and save information

In [1]:
import numpy as np

WALL   = 1
EMPTY  = 0
ELF    = 2
GOBLIN = 3

def readMap(filename):
    with open(filename) as f:
        lines = [ l.strip("\n") for l in f.readlines() ]
    elfs = []
    gobl = []
    grid = np.zeros((len(lines),len(lines[0])),dtype=int)
    y = 0
    for l in lines:
        x = 0
        for c in l:
            if c=="#":
                grid[y,x] = WALL
            elif c=="E":
                grid[y,x] = ELF
                elfs.append((x,y))
            elif c=="G":
                grid[y,x] = GOBLIN
                gobl.append((x,y))
            x +=1
        y+=1
    return grid,elfs,gobl

### Shortest paths from point A to point B

In [36]:
from queue import Queue, PriorityQueue
from collections import defaultdict

def empty(p,grid):
    '''Neighboring empty positions'''
    xp,yp = p
    return [ (xp+i,yp+j) for i,j in ((-1,0),(0,-1),(0,+1),(+1,0))  
            if 0<=xp+i<len(grid[0]) and 0<=yp+j<len(grid) and grid[yp+j][xp+i]==EMPTY ]

def shortestPath(s,e,grid):
    '''
        Return one shortest path from s to e assuming walls, elfs and goblins are not traversable. 
        s can be a unit (elf or goblin) but e is supposed to be empty.
        BFS algorithm.
    '''
    q = Queue()
    q.put([s])
    while len(q.queue):
        path = q.get()
        if path[-1] == e: # found end
            return path
        for n in empty(path[-1],grid):
            if n not in path:
                pathnew = path+[n]
                q.put(pathnew)
    return []

def shortestPathL(s,e,grid):
    '''
        Return lenght of one shortest path from s to e assuming walls, elfs and goblins are not traversable. 
        s can be a unit (elf or goblin) but e is supposed to be empty.
        BFS algorithm.
    '''
    q = Queue()
    q.put((s,0))
    visited = defaultdict(bool)
    visited[s] = True
    while len(q.queue):
        p,l = q.get()
        if p == e: # found end
            return l
        for n in empty(p,grid):
            if not visited[n]:
                q.put((n,l+1))
                visited[n] = True    
    return -1

In [37]:
def plotGrid(grid):
    G = {}
    G[WALL]   = "#"
    G[EMPTY]  = "."
    G[ELF]    = "E"
    G[GOBLIN] = "G"
    for y in range(len(grid)):
        for x in range(len(grid[0])):
            print(G[grid[y,x]],end="")
        print()

grid,elfs,gobl = readMap("data/day15test0.txt")

%matplotlib inline
import matplotlib.pyplot as plt
#plt.imshow(grid)

plotGrid(grid)

#######
#E..G.#
#...#.#
#.G.#G#
#######


### Choose target and step toward it

In [40]:
def chooseTarget(e,targets,grid):
    # In range position around targets
    inrange = [] 
    for t in targets:
        for n in empty(t,grid):
            if n not in inrange:
                inrange.append(n)
    # Reachable targets, grouped by distances
    dists = defaultdict(lambda: []) 
    for i in inrange:
        lp = shortestPathL(e,i,grid)
        if lp!=-1:
            dists[lp].append(i)
    # Closest target. Sorting by reading order to discriminate between multiples with same distance
    if len(dists.keys()):
        nearests = sorted( dists[ min(dists.keys()) ] , key=lambda x: (x[1],x[0]) )
        return nearests[0]
    else:
        return None
    
def chooseTargetPath(e,targets,grid):
    t = chooseTarget(e,targets,grid)
    if t!=None: # has a reachable target
        nn = empty(e,grid)
        if len(nn)==1: # only a possible first step toward target
            return(nn[0])
        else:
            shortest = defaultdict(lambda: [])
            for n in empty(e,grid): # possible first steps toward target
                ln = shortestPathL(n,t,grid)
                if ln!=-1:
                    shortest[ln+1].append(n)
            return sorted(shortest[min(shortest.keys())] , key=lambda x: (x[1],x[0]) )[0]
    else:
        return None

In [41]:
# Targets:      In range:     Reachable:    Nearest:      Chosen:  
# #######       #######       #######       #######       #######  
# #E..G.#       #E.?G?#       #E.@G.#       #E.!G.#       #E.+G.#  
# #...#.#  -->  #.?.#?#  -->  #.@.#.#  -->  #.!.#.#  -->  #...#.#  
# #.G.#G#       #?G?#G#       #@G@#G#       #!G.#G#       #.G.#G#  
# #######       #######       #######       #######       #######  

grid,elfs,gobl = readMap("data/day15test0.txt")
plotGrid(grid)
print(chooseTarget((1,1),gobl,grid))

#######
#E..G.#
#...#.#
#.G.#G#
#######
(3, 1)


In [42]:
grid,elfs,gobl = readMap("data/day15test1.txt")
plotGrid(grid)
chooseTargetPath((2,1),gobl,grid)

#######
#.E...#
#.....#
#...G.#
#######


(3, 1)

### Movement example

In [82]:
Race = {}
Race[GOBLIN] = "Goblin"
Race[ELF] = "Elf"

class Unit:
    def __init__(self,r,p,hp=200,sp=3):
        self.race  = r
        self.pos   = p
        self.hp    = hp
        self.power = sp

def getUnits(elfs,gobl,goblpower=3,elfpower=3):
    '''Initialise unit list'''
    units = []
    for g in gobl:
        units.append(Unit(GOBLIN,g,200,goblpower))
    for e in elfs:
        units.append(Unit(ELF,e,200,elfpower))
    return units

def targetRange(p,grid):
    '''
        return coordinate of anything that is not WALL around unit.
        To be tested against Elfs' or Goblins' coordinate lists.
    '''
    xp,yp = p
    return [ (xp+i,yp+j) for i,j in ((-1,0),(0,-1),(0,+1),(+1,0))  
            if 0<=xp+i<len(grid[0]) and 0<=yp+j<len(grid) and grid[yp+j][xp+i]!=WALL ]

def canAttack(U,gobl,elfs,grid):
    for t in targetRange(U.pos,grid):
        if U.race==GOBLIN and t in elfs:
            return True
        elif U.race==ELF and t in gobl:
            return True
    return False

from copy import deepcopy

def movementTest(_grid,_elfs,_gobl,rmax=3):

    grid = deepcopy(_grid)
    elfs = deepcopy(_elfs)
    gobl = deepcopy(_gobl)
    units = getUnits(elfs,gobl)    

    print("Initially:")
    plotGrid(grid)
    
    r = 0 # round number
    
    while r<rmax:

        print("\nRound {}...".format(r+1))
        
        units = sorted(units, key=lambda U: (U.pos[1],U.pos[0]))

        for U in units:

            # Check whether there's en enemy in range for attack
            attack = canAttack(U,gobl,elfs,grid)
            
            # If no enemy to attack, try to move
            if not attack:
                p = None
                if U.race==GOBLIN:            
                    p = chooseTargetPath(U.pos,elfs,grid)
                else:
                    p = chooseTargetPath(U.pos,gobl,grid)
                if p: # can move
                    # move
                    xn,yn = p     # new position coord
                    xu,yu = U.pos # old position coord
                    # update grid
                    grid[yu][xu]=EMPTY
                    grid[yn][xn]=U.race
                    # update list of positions
                    if U.race==GOBLIN: 
                        gobl.pop(gobl.index(U.pos))
                        gobl.append(p)
                    else:
                        elfs.pop(elfs.index(U.pos))
                        elfs.append(p)  
                    # update unit position
                    U.pos = p

            else: # attack logic will go here
                print("{:6s} at {} would attack!".format(Race[U.race],U.pos))

        r+=1
        print("After {} round(s):".format(r))
        plotGrid(grid)

In [83]:
grid,elfs,gobl = readMap("data/day15test2.txt")
movementTest(grid,elfs,gobl)

Initially:
#########
#G..G..G#
#.......#
#.......#
#G..E..G#
#.......#
#.......#
#G..G..G#
#########

Round 1...
After 1 round(s):
#########
#.G...G.#
#...G...#
#...E..G#
#.G.....#
#.......#
#G..G..G#
#.......#
#########

Round 2...
Goblin at (4, 2) would attack!
Elf    at (4, 3) would attack!
After 2 round(s):
#########
#..G.G..#
#...G...#
#.G.E.G.#
#.......#
#G..G..G#
#.......#
#.......#
#########

Round 3...
Goblin at (4, 2) would attack!
Elf    at (4, 3) would attack!
After 3 round(s):
#########
#.......#
#..GGG..#
#..GEG..#
#G..G...#
#......G#
#.......#
#.......#
#########


### Combat

In [168]:
def combat(_grid,_elfs,_gobl,elfpower=3,part=1,verbose=False,silent=False,rmax=-1,progress=False):

    grid = deepcopy(_grid)
    elfs = deepcopy(_elfs)
    gobl = deepcopy(_gobl)
    units = getUnits(elfs,gobl,elfpower=elfpower)

    if not silent:
        print("--- Initially ---\n")
        plotGrid(grid)
    
    fr = 0 # number of full rounds
    r = 0 # round number
    
    if progress and not silent:
        print("\nRounds:",end=" ")
    
    while True:

        r+=1
        
        if rmax!=-1 and r>=rmax:
            break

        if verbose and not silent:
            print("\n--- Round {} begins ---\n".format(r))
        
        if progress and not silent:
            print(r,end=" ")
            
        units = sorted(units, key=lambda U: (U.pos[1],U.pos[0]))
        
        np = len([ U for U in units if U.hp>0 ])
        ip = 0
        isFinished = False
        
        if verbose and not silent:
            print("{} active units".format(np))
        
        for U in units:

            # Ignore dead units
            if U.hp<=0:
                continue
            
            ip+=1
            
            # Check whether there's en enemy in range for attack
            attack = canAttack(U,gobl,elfs,grid)
            
            # If no enemy to attack, try to move
            hasMoved = False
            if not attack:
                p = None
                if U.race==GOBLIN:            
                    p = chooseTargetPath(U.pos,elfs,grid)
                else:
                    p = chooseTargetPath(U.pos,gobl,grid)
                if p: # can move
                    # move
                    xn,yn = p     # new position coord
                    xu,yu = U.pos # old position coord
                    # update grid
                    grid[yu][xu]=EMPTY
                    grid[yn][xn]=U.race
                    # update list of positions
                    if U.race==GOBLIN: 
                        gobl.pop(gobl.index(U.pos))
                        gobl.append(p)
                    else:
                        elfs.pop(elfs.index(U.pos))
                        elfs.append(p)  
                    # update unit position
                    U.pos = p
                    hasMoved = True
            
            # Check again whether unit can attack after movement
            if hasMoved:
                attack = canAttack(U,gobl,elfs,grid)

            # Attack
            if attack:
                # There might be multiple targets, in case choose that with lowest HP
                targets = []
                for t in targetRange(U.pos,grid):
                    if U.race==GOBLIN and t in elfs:
                        targets.append(t)
                    elif U.race==ELF and t in gobl:
                        targets.append(t)
                if verbose and not silent:
                    print("{:6s} at {} (HP: {}) has target(s) in {} ...".format(Race[U.race],U.pos,U.hp,targets), end=" ")
                # attack weakest one
                Td = defaultdict(lambda: [])
                for w in [ W for W in units if W.pos in targets and W.hp>0 ]:
                    Td[w.hp].append(w)
                T = sorted( Td[min(Td.keys())], key=lambda T: (T.pos[1],T.pos[0]) )
                if verbose and not silent:
                    print("Attacking {}".format(T[0].pos))
                T[0].hp -= U.power
                # if target is dead, remove it from list and grid
                if T[0].hp <= 0:
                    ### Part 2 check
                    if part==2 and T[0].race==ELF: # at least one elf died!
                        print("An elf died! Power {} not enough".format(elfpower))
                        return -1,-1,-1,grid
                    ###
                    if verbose and not silent:
                        print("{:6s} at {} (HP: {}) is dead!".format(Race[T[0].race],T[0].pos,T[0].hp))
                    if T[0].race==GOBLIN:
                        gobl.pop(gobl.index(T[0].pos))
                    elif T[0].race==ELF:
                        elfs.pop(elfs.index(T[0].pos))
                    xt,yt = T[0].pos
                    grid[yt][xt] = EMPTY
                        
            # Check for status after each attack
            if len(elfs)==0:
                if progress and not silent: print()
                if not silent:
                    print("\nAll Elfs dead during round {}!\n".format(r))
                    plotGrid(grid)
                isFinished = True
                break

            if len(gobl)==0:
                if progress and not silent: print()
                if not silent:
                    print("\nAll Goblins dead during round {}!\n".format(r))
                    plotGrid(grid)
                    print(np,ip)
                isFinished = True
                break      
        
        # check if round is "full"
        if ip==np:
            fr = r
        else:
            fr = r-1
            
        if not isFinished: # nobody won yet
            # purge possible death units after round
            #units = [ U for U in sorted(units, key=lambda U: (U.pos[1],U.pos[0])) if U.hp > 0 ]
            if verbose and not silent:
                print("\n--- After {} round ---\n".format(r))
                plotGrid(grid)
                print()
                units = sorted(units, key=lambda U: (U.pos[1],U.pos[0]))
                for U in units:
                    if U.hp > 0:
                        print("{:6s} {} HP: {}".format(Race[U.race],U.pos,U.hp))
        else: # we have a winner team!
            break

    if progress and not silent:
        print()
            
    # compute score
    score = 0
    for U in units:
        if U.hp>0:
            score += max(0,U.hp)
    outcome = score * fr
    
    if not silent:
        print()
        for U in units:
            if U.hp>0:
                print("{:6s} {} HP: {}".format(Race[U.race],U.pos,U.hp))
        print("\nScore = {} after {} full rounds, Outcome = {}".format(score,fr,outcome))

    return score,outcome,fr,grid

In [169]:
grid,elfs,gobl = readMap("data/day15test3.txt")

score, outcome, r, grid = combat(grid,elfs,gobl)

--- Initially ---

#######
#.G...#
#...EG#
#.#.#G#
#..G#E#
#.....#
#######

All Elfs dead during round 47!

#######
#G....#
#.G...#
#.#.#G#
#...#.#
#....G#
#######

Goblin (1, 1) HP: 200
Goblin (2, 2) HP: 131
Goblin (5, 3) HP: 59
Goblin (5, 5) HP: 200

Score = 590 after 47 full rounds, Outcome = 27730


In [170]:
grid,elfs,gobl = readMap("data/day15test4.txt")
score, outcome, r, grid = combat(grid,elfs,gobl)

--- Initially ---

#######
#G..#E#
#E#E.E#
#G.##.#
#...#E#
#...E.#
#######

All Goblins dead during round 38!

#######
#...#E#
#E#...#
#.E##.#
#E..#E#
#.....#
#######
6 4

Elf    (5, 1) HP: 200
Elf    (1, 2) HP: 197
Elf    (2, 3) HP: 185
Elf    (1, 4) HP: 200
Elf    (5, 4) HP: 200

Score = 982 after 37 full rounds, Outcome = 36334


In [171]:
grid,elfs,gobl = readMap("data/day15test5.txt")
score, outcome, r, grid = combat(grid,elfs,gobl)

--- Initially ---

#######
#E..EG#
#.#G.E#
#E.##E#
#G..#.#
#..E#.#
#######

All Goblins dead during round 47!

#######
#.E.E.#
#.#E..#
#E.##.#
#.E.#.#
#...#.#
#######
6 1

Elf    (2, 1) HP: 164
Elf    (4, 1) HP: 197
Elf    (3, 2) HP: 200
Elf    (1, 3) HP: 98
Elf    (2, 4) HP: 200

Score = 859 after 46 full rounds, Outcome = 39514


In [172]:
grid,elfs,gobl = readMap("data/day15test6.txt")
score, outcome, r, grid = combat(grid,elfs,gobl)

--- Initially ---

#######
#E.G#.#
#.#G..#
#G.#.G#
#G..#.#
#...E.#
#######

All Elfs dead during round 36!

#######
#G.G#.#
#.#G..#
#..#..#
#...#G#
#...G.#
#######

Goblin (1, 1) HP: 200
Goblin (3, 1) HP: 98
Goblin (3, 2) HP: 200
Goblin (5, 4) HP: 95
Goblin (4, 5) HP: 200

Score = 793 after 35 full rounds, Outcome = 27755


In [173]:
grid,elfs,gobl = readMap("data/day15test7.txt")
score, outcome, r, grid = combat(grid,elfs,gobl)

--- Initially ---

#######
#.E...#
#.#..G#
#.###.#
#E#G#G#
#...#G#
#######

All Elfs dead during round 55!

#######
#.....#
#.#G..#
#.###.#
#.#.#.#
#G.G#G#
#######

Goblin (3, 2) HP: 200
Goblin (1, 5) HP: 98
Goblin (3, 5) HP: 38
Goblin (5, 5) HP: 200

Score = 536 after 54 full rounds, Outcome = 28944


In [174]:
grid,elfs,gobl = readMap("data/day15test8.txt")
score, outcome, r, grid = combat(grid,elfs,gobl,progress=True)

--- Initially ---

#########
#G......#
#.E.#...#
#..##..G#
#...##..#
#...#...#
#.G...G.#
#.....G.#
#########

Rounds: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 

All Elfs dead during round 21!

#########
#.G.....#
#G.G#...#
#.G##...#
#...##..#
#.G.#...#
#.......#
#.......#
#########


Goblin (2, 1) HP: 137
Goblin (1, 2) HP: 200
Goblin (3, 2) HP: 200
Goblin (2, 3) HP: 200
Goblin (2, 5) HP: 200

Score = 937 after 20 full rounds, Outcome = 18740


In [175]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [176]:
grid,elfs,gobl = readMap("data/input15.txt")

score, outcome, r, grid = combat(grid,elfs,gobl,progress=True)

--- Initially ---

################################
######################........##
####################...........#
##############G.G..G.#.......#.#
#############.....G...#....###.#
############..##.G.............#
#############...#...GG......####
#############......G......######
##############G....EG.....######
#############.......G.....######
############.....G......#.######
###########......E...G.#########
##########....#####......#######
##########G..#######......######
######......#########....#######
#####....G..#########....#######
###.......#.#########....#######
###.G.....#.#########E...#######
#........##.#########E...#######
#.......###..#######...E.#######
#.#.#.........#####.......######
#.###.#.###.G..............#####
####.....###..........E.##.#####
#......G####.E..........########
###..G..####...........####..###
####..########..E......###...###
###..............#...E...#.#####
##.........##....##........#####
#.......#.####.........#########
#...##G.##########....E#

## Part 2

In [178]:
grid,elfs,gobl = readMap("data/day15test3.txt")
score, outcome, r, grid = combat(grid,elfs,gobl,part=2,elfpower=15)

--- Initially ---

#######
#.G...#
#...EG#
#.#.#G#
#..G#E#
#.....#
#######

All Goblins dead during round 30!

#######
#..E..#
#...E.#
#.#.#.#
#...#.#
#.....#
#######
3 1

Elf    (3, 1) HP: 158
Elf    (4, 2) HP: 14

Score = 172 after 29 full rounds, Outcome = 4988


In [201]:
def increaseElfPower(grid,elfs,gobl,verbose=True):
    ep = 4
    while True:
        print("Trying elf power = {:3d} ... ".format(ep),end="")
        score, outcome, fr, gridend = combat(grid,elfs,gobl,part=2,elfpower=ep,silent=True)
        if score!=-1:
            print("Finally successfull!")
            print("\nElf power = {} => Score = {} after {} full rounds, Outcome = {}".format(ep,score,fr,outcome))
            return
        ep += 1

In [190]:
grid,elfs,gobl = readMap("data/day15test3.txt")
increaseElfPower(grid,elfs,gobl)

Trying elf power =   4 ... An elf died! Power 4 not enough
Trying elf power =   5 ... An elf died! Power 5 not enough
Trying elf power =   6 ... An elf died! Power 6 not enough
Trying elf power =   7 ... An elf died! Power 7 not enough
Trying elf power =   8 ... An elf died! Power 8 not enough
Trying elf power =   9 ... An elf died! Power 9 not enough
Trying elf power =  10 ... An elf died! Power 10 not enough
Trying elf power =  11 ... An elf died! Power 11 not enough
Trying elf power =  12 ... An elf died! Power 12 not enough
Trying elf power =  13 ... An elf died! Power 13 not enough
Trying elf power =  14 ... An elf died! Power 14 not enough
Trying elf power =  15 ... Finally successfull!

Elf power = 15 => Score = 172 after 29 full rounds, Outcome = 4988


In [191]:
grid,elfs,gobl = readMap("data/day15test4.txt")
increaseElfPower(grid,elfs,gobl)

Trying elf power =   4 ... Finally successfull!

Elf power = 4 => Score = 1038 after 28 full rounds, Outcome = 29064


In [192]:
grid,elfs,gobl = readMap("data/day15test5.txt")
increaseElfPower(grid,elfs,gobl)

Trying elf power =   4 ... Finally successfull!

Elf power = 4 => Score = 948 after 33 full rounds, Outcome = 31284


In [193]:
grid,elfs,gobl = readMap("data/day15test6.txt")
increaseElfPower(grid,elfs,gobl)

Trying elf power =   4 ... An elf died! Power 4 not enough
Trying elf power =   5 ... An elf died! Power 5 not enough
Trying elf power =   6 ... An elf died! Power 6 not enough
Trying elf power =   7 ... An elf died! Power 7 not enough
Trying elf power =   8 ... An elf died! Power 8 not enough
Trying elf power =   9 ... An elf died! Power 9 not enough
Trying elf power =  10 ... An elf died! Power 10 not enough
Trying elf power =  11 ... An elf died! Power 11 not enough
Trying elf power =  12 ... An elf died! Power 12 not enough
Trying elf power =  13 ... An elf died! Power 13 not enough
Trying elf power =  14 ... An elf died! Power 14 not enough
Trying elf power =  15 ... Finally successfull!

Elf power = 15 => Score = 94 after 37 full rounds, Outcome = 3478


In [194]:
grid,elfs,gobl = readMap("data/day15test7.txt")
increaseElfPower(grid,elfs,gobl)

Trying elf power =   4 ... An elf died! Power 4 not enough
Trying elf power =   5 ... An elf died! Power 5 not enough
Trying elf power =   6 ... An elf died! Power 6 not enough
Trying elf power =   7 ... An elf died! Power 7 not enough
Trying elf power =   8 ... An elf died! Power 8 not enough
Trying elf power =   9 ... An elf died! Power 9 not enough
Trying elf power =  10 ... An elf died! Power 10 not enough
Trying elf power =  11 ... An elf died! Power 11 not enough
Trying elf power =  12 ... Finally successfull!

Elf power = 12 => Score = 166 after 39 full rounds, Outcome = 6474


In [200]:
grid,elfs,gobl = readMap("data/day15test8.txt")
increaseElfPower(grid,elfs,gobl)

Trying elf power =   4 ... An elf died! Power 4 not enough
Trying elf power =   5 ... An elf died! Power 5 not enough
Trying elf power =   6 ... An elf died! Power 6 not enough
Trying elf power =   7 ... An elf died! Power 7 not enough
Trying elf power =   8 ... An elf died! Power 8 not enough
Trying elf power =   9 ... An elf died! Power 9 not enough
Trying elf power =  10 ... An elf died! Power 10 not enough
Trying elf power =  11 ... An elf died! Power 11 not enough
Trying elf power =  12 ... An elf died! Power 12 not enough
Trying elf power =  13 ... An elf died! Power 13 not enough
Trying elf power =  14 ... An elf died! Power 14 not enough
Trying elf power =  15 ... An elf died! Power 15 not enough
Trying elf power =  16 ... An elf died! Power 16 not enough
Trying elf power =  17 ... An elf died! Power 17 not enough
Trying elf power =  18 ... An elf died! Power 18 not enough
Trying elf power =  19 ... An elf died! Power 19 not enough
Trying elf power =  20 ... An elf died! Power 

In [199]:
grid,elfs,gobl = readMap("data/input15.txt")
increaseElfPower(grid,elfs,gobl)

Trying elf power =   4 ... An elf died! Power 4 not enough
Trying elf power =   5 ... An elf died! Power 5 not enough
Trying elf power =   6 ... An elf died! Power 6 not enough
Trying elf power =   7 ... An elf died! Power 7 not enough
Trying elf power =   8 ... An elf died! Power 8 not enough
Trying elf power =   9 ... An elf died! Power 9 not enough
Trying elf power =  10 ... An elf died! Power 10 not enough
Trying elf power =  11 ... An elf died! Power 11 not enough
Trying elf power =  12 ... An elf died! Power 12 not enough
Trying elf power =  13 ... An elf died! Power 13 not enough
Trying elf power =  14 ... An elf died! Power 14 not enough
Trying elf power =  15 ... An elf died! Power 15 not enough
Trying elf power =  16 ... An elf died! Power 16 not enough
Trying elf power =  17 ... An elf died! Power 17 not enough
Trying elf power =  18 ... An elf died! Power 18 not enough
Trying elf power =  19 ... An elf died! Power 19 not enough
Trying elf power =  20 ... An elf died! Power 